In [1]:
import numpy as np
import matplotlib.cm as cmap
import matplotlib.pyplot as plt
import time
import os.path
import scipy
import pickle as pickle
from struct import unpack
from brian2 import *
import brian2
from brian2tools import *
from PIL import Image
import scipy.signal as signal
import warnings
warnings.simplefilter('ignore')
import random
import gc

In [2]:
def get_surrounding(coordinate, r, inner, length):
    y = coordinate[0]
    x = coordinate[1]
    
    x_min = 0 if (x-r < 0) else (x-r)
    x_max = (length-1) if (x+r > length-1) else (x+r)
    y_min = 0 if (y-r < 0) else (y-r)
    y_max = (length-1) if (y+r > length-1) else (y+r)
    
    coordinates = []
    for i in range(y_min, y_max+1):
        for j in range(x_min, x_max+1):
            if (i==y and j==x and not inner):
                continue
            coordinates.append((i,j))
            
    return coordinates

In [3]:
# initial random weights
dataPath = './initial_weights/'

field_size = 2                     # 2/3/4/5/6
w_s = field_size*1                # simple cell width
l_s = field_size*2                # simple_cell_length

c_length = 10
r_length = (c_length*l_s)   # (c_length*2*w_s)
orientations = 4

n_LGN = r_length*r_length
n_L4  = c_length*c_length*orientations
n_L3  = n_L4
n_L2  = n_L3

weight = {}
weight['LGN_L4'] = 10
weight['L4_L4'] = 30
weight['L4_L3_round'] = 2
weight['L4_L3_direct'] = 30
weight['L3_L3'] = 0.5
weight['L3_L2'] = 8   # 10
weight['L2_L3'] = 30
# weight['L2_L3'] = 0.001
weight['L3_L4_direct'] = 15
weight['L3_L4_round'] = 1

h_index = 0
v_index = 1
d1_index = 2
d2_index = 3

In [4]:
# LGN ON >> L4 horizontal   (NF connection)
# LGN OFF >> L4 horizontal  (FN connection)

weightList = np.random.random((n_LGN, n_L4))
weightList += weight['LGN_L4']

weights_NF = []
weights_FN = []

for i in range(0, c_length):
    for j in range(0, c_length):
        
        # horizontal selectivity
        for p in range(0, w_s):
            for q in range(0, l_s):
                x_NF = (i*2*w_s+p) * r_length + (j*l_s+q)
                x_FN = ((i*2+1)*w_s+p) * r_length + (j*l_s+q)
                y = (i * c_length + j) * orientations + h_index

                weights_NF.append((x_NF, y, weightList[x_NF,y]))
                weights_FN.append((x_FN, y, weightList[x_FN,y]))
            
        # vertical selectivity
        for p in range(0, l_s):
            for q in range(0, w_s):
                x_NF = (i*l_s+p) * r_length + (j*2*w_s+q)
                x_FN = (i*l_s+p) * r_length + ((j*2+1)*w_s+q)
                y = (i * c_length + j) * orientations + v_index

                weights_NF.append((x_NF, y, weightList[x_NF,y]))
                weights_FN.append((x_FN, y, weightList[x_FN,y]))
          
        # diagonal 45 selectivity
        x_NF_list = [
            (i*l_s+0)*r_length + (j*2*w_s+0), (i*l_s+0)*r_length + (j*2*w_s+1), (i*l_s+0)*r_length + (j*2*w_s+2),
            (i*l_s+1)*r_length + (j*2*w_s+0), (i*l_s+1)*r_length + (j*2*w_s+1),
            (i*l_s+2)*r_length + (j*2*w_s+0)
        ]
        x_FN_list = [
            (i*l_s+1)*r_length + (j*2*w_s+3),
            (i*l_s+2)*r_length + (j*2*w_s+2), (i*l_s+2)*r_length + (j*2*w_s+3),
            (i*l_s+3)*r_length + (j*2*w_s+1), (i*l_s+3)*r_length + (j*2*w_s+2), (i*l_s+3)*r_length + (j*2*w_s+3)
        ]
        y = (i * c_length + j) * orientations + d1_index

        for x_NF in x_NF_list:
            weights_NF.append((x_NF, y, weightList[x_NF,y]))
        for x_FN in x_FN_list:
            weights_FN.append((x_FN, y, weightList[x_FN,y]))
            
        # diagonal 135 selectivity
        x_NF_list = [
            (i*l_s+0)*r_length + (j*2*w_s+1), (i*l_s+0)*r_length + (j*2*w_s+2), (i*l_s+0)*r_length + (j*2*w_s+3),
            (i*l_s+1)*r_length + (j*2*w_s+2), (i*l_s+1)*r_length + (j*2*w_s+3),
            (i*l_s+2)*r_length + (j*2*w_s+3)
        ]
        x_FN_list = [
            (i*l_s+1)*r_length + (j*2*w_s+0),
            (i*l_s+2)*r_length + (j*2*w_s+0), (i*l_s+2)*r_length + (j*2*w_s+1),
            (i*l_s+3)*r_length + (j*2*w_s+0), (i*l_s+3)*r_length + (j*2*w_s+1), (i*l_s+3)*r_length + (j*2*w_s+2)
        ]
        y = (i * c_length + j) * orientations + d2_index

        for x_NF in x_NF_list:
            weights_NF.append((x_NF, y, weightList[x_NF,y]))
        for x_FN in x_FN_list:
            weights_FN.append((x_FN, y, weightList[x_FN,y]))   
        
        # garbage collect
        gc.collect()
        
np.save(dataPath + 'LGN_L4_NF', weights_NF)
np.save(dataPath + 'LGN_L4_FN', weights_FN)

In [5]:
# L4 >> L4
# one to one

weightList = np.random.random(n_L4)
weightList += weight['L4_L4']
weights = [(i, i, weightList[i]) for i in range(n_L4)]
np.save(dataPath + 'L4_L4', weights)

In [6]:
# L4 >> L2/3 (L3)
# many to one (circle + direct)
# excitatory vertical forward connections

weight_direct_list  = np.random.random(n_L4)
weight_direct_list += weight['L4_L3_direct']

weights = [(k, k, weight_direct_list[k]) for k in range(n_L4)]

# weights = []

weight_round_list   = np.random.random((n_L4, n_L3))
weight_round_list  += weight['L4_L3_round']

for i in range(0, c_length):                   # y axis
    for j in range(0, c_length):               # x axis
        for p,q in get_surrounding((i,j), 1, False, c_length):
            
            for o_index in range(orientations):
                source = ( p * c_length + q ) * orientations + o_index
                target = ( i * c_length + j ) * orientations + o_index
                
                weights.append((source, target, weight_round_list[source,target]))
                 
np.save(dataPath + 'L4_L3', weights)

In [7]:
# L2/3 (L3) >> L2/3 (L3)
# many to one (circle)
# excitatory horizontal connections
# excite same orientation selectivities

weightList  = np.zeros((n_L3, n_L3))
weightList += weight['L3_L3']

# weights = [(i, j, weightList[i,j]) for i in range(n_L3) for j in range(n_L3)]
weights = []

for i in range(0, c_length):                   # y axis
    for j in range(0, c_length):               # x axis
        for p,q in get_surrounding((i,j), 1, False, c_length):
            
            for s_o_index in range(orientations):
                for t_o_index in range(orientations):
                    target = ( p * c_length + q ) * orientations + t_o_index
                    source = ( i * c_length + j ) * orientations + s_o_index
                    value = weightList[source,target] + np.random.random() * 0.01
                    weights.append((source, target, value))
                 
np.save(dataPath + 'L3_L3', weights)

In [8]:
# L2/3 (L3) >> L2/3 (L2)
# many to one (circle)
# excitatory horizontal connections
# inhibit un-compatible orientations selectivities

weightList  = np.zeros((n_L3, n_L2))
weightList += weight['L3_L2']

# weights = [(i, j, weightList[i,j]) for i in range(n_L3) for j in range(n_L2)]
weights = []

for i in range(0, c_length):                   # y axis
    for j in range(0, c_length):               # x axis
        for p,q in get_surrounding((i,j), 0, True, c_length):
            
            for s_o_index in range(orientations):           # source orientations
                for t_o_index in range(orientations):       # target orientations
                    target = ( p * c_length + q ) * orientations + t_o_index
                    source = ( i * c_length + j ) * orientations + s_o_index
                    value = weightList[source,target] + np.random.random() * 0.01
                    weights.append((source, target, value))
                 
np.save(dataPath + 'L3_L2', weights)

In [9]:
# L2/3 (L2) >> L2/3 (L3)
# one to one
# inhibitory vertical connections

weightList = np.random.random(n_L2)
weightList += weight['L2_L3']
weights = [(i, i, weightList[i]) for i in range(n_L2)]
np.save(dataPath + 'L2_L3', weights)

In [10]:
# L2/3 (L3) >> L4
# many to one (circle + direct)
# excitatory vertical feedback connections
gc.collect()

weight_direct_list  = np.random.random(n_L3)
weight_direct_list += weight['L3_L4_direct']

weights = [(k, k, weight_direct_list[k]) for k in range(n_L3)]

# weights = []

weight_round_list   = np.random.random((n_L3, n_L4)) 
weight_round_list  += weight['L3_L4_round']

for i in range(0, c_length):                   # y axis
    for j in range(0, c_length):               # x axis
        for p,q in get_surrounding((i,j), 1, False, c_length):
            
            for o_index in range(orientations):
                target = ( p * c_length + q ) * orientations + o_index
                source = ( i * c_length + j ) * orientations + o_index
                
                weights.append((source, target, weight_round_list[source,target]))
                 
np.save(dataPath + 'L3_L4', weights)

In [11]:
# len(weights)

In [12]:
# weights[0]

In [ ]:
get_surrounding((5,5), 0, True, 10)

In [ ]:
np.random.random()

In [ ]:
# vertical line generator
grid = np.random